Aufgabe 2b old

In [ ]:
# Settings:
start_time = "2019-01-03"
end_time = "2019-01-10"
time_steps = "h"

df = charging_data_cleaned[(charging_data_cleaned["connectionTime"] >= start_time) & (charging_data_cleaned["connectionTime"] < end_time)]

# Convert input data to DataFrame
df.loc[:, "connectionTime"] = pd.to_datetime(df["connectionTime"]).dt.round(time_steps)
df.loc[:, "disconnectTime"] = pd.to_datetime(df["disconnectTime"]).dt.round(time_steps)
df.loc[:, "doneChargingTime"] = pd.to_datetime(df["doneChargingTime"]).dt.round(time_steps)

# Define the range of hours to analyze
start_time = df["connectionTime"].min()
end_time = df["disconnectTime"].max()
hourly_range = pd.date_range(start=start_time, end=end_time, freq=time_steps)

# Create an empty result DataFrame
result = pd.DataFrame({"datetime": hourly_range})

# Merge the sessions into the hourly range
hourly_sessions = []
for _, row in df.iterrows():
    session_hours = pd.date_range(start=row["connectionTime"], end=row["disconnectTime"], freq=time_steps)
    
    # Calculate kWh delivered per hour with protection against zero duration
    duration_hours = (row["disconnectTime"] - row["connectionTime"]).total_seconds() / 3600.0
    if duration_hours > 0:
        kWh_per_hour = row["kWhDelivered"] / duration_hours
    else:
        kWh_per_hour = 0  # No duration, no charging
    
    for hour in session_hours:
        hourly_sessions.append({
            "datetime": hour,
            "stationID": row["stationID"],
            "isCharging": int(row["doneChargingTime"] >= hour),
            "kWhDelivered": kWh_per_hour
        })

# Create a DataFrame for hourly sessions
hourly_df = pd.DataFrame(hourly_sessions)

# Aggregate data to calculate KPIs
aggregated = hourly_df.groupby("datetime").agg(
    blockedStations=("stationID", "nunique"),  # Count unique station IDs
    chargingStations=("isCharging", "sum"),   # Sum charging stations
    kWhDelivered=("kWhDelivered", "sum")      # Sum kWh delivered
).reset_index()

aggregated["blockedStations"] = aggregated["blockedStations"] / 107
aggregated["chargingStations"] = aggregated["chargingStations"] / 107
aggregated["idleTime"] = aggregated["chargingStations"] / aggregated["blockedStations"]

# Merge the results back to the result DataFrame
result = result.merge(aggregated, on="datetime", how="left").fillna(0)

# Keep only relevant columns
result = result[["datetime", "blockedStations", "chargingStations", "kWhDelivered", "idleTime"]]

# Display the result
display(result.head())

# Display the result

plt.figure(figsize=(12, 6))
sns.lineplot(data=result, x='datetime', y='blockedStations', marker='o', color='tab:blue')
sns.lineplot(data=result, x='datetime', y='chargingStations', marker='o', color='tab:orange')
plt.title('Utilization Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Utilization Rate (%)')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.lineplot(data=result, x='datetime', y='idleTime', marker='o', color='green')
plt.title('Hourly KWh delivered Over Time')
plt.xlabel('Date')
plt.ylabel('KWh delivered')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 6))
sns.lineplot(data=result, x='datetime', y='kWhDelivered', marker='o', color='green')
plt.title('Hourly KWh delivered Over Time')
plt.xlabel('Date')
plt.ylabel('KWh delivered')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

In [ ]:
# Task (b): Key Performance Indicators (KPIs)
# Define KPIs
charging_data_2019 = charging_data_cleaned[
    (charging_data_cleaned['connectionTime'].dt.year == 2019) &
    (charging_data_cleaned['connectionTime'].dt.month == 1)
]

# Schritt 1: Spalten vorbereiten
charging_data_2019['hour'] = charging_data_2019['connectionTime'].dt.hour

# Gesamtzeit im Betrachtungszeitraum (pro Tag)
total_stations = charging_data_2019['stationID'].nunique()
total_minutes_per_day = total_stations * 24 * 60

# Gesamtzeit, Ladezeit und Leerlaufzeit berechnen
charging_data_2019['usage_minutes'] = (charging_data_2019['disconnectTime'] - charging_data_2019['connectionTime']).dt.total_seconds() / 60
charging_data_2019['charging_minutes'] = (charging_data_2019['doneChargingTime'] - charging_data_2019['connectionTime']).dt.total_seconds() / 60
charging_data_2019['idle_minutes'] = (charging_data_2019['disconnectTime'] - charging_data_2019['doneChargingTime']).dt.total_seconds() / 60

# Aggregation nach Datum
daily_aggregates = charging_data_2019.groupby('hour').agg(
    used_minutes=('usage_minutes', 'sum'),
    charging_minutes=('charging_minutes', 'sum'),
    idle_minutes=('idle_minutes', 'sum'),
    peak_hour_sessions=('kWhDelivered', 'sum')
)


# KPIs berechnen
daily_aggregates['utilization_rate'] = (daily_aggregates['used_minutes'] / total_minutes_per_day) * 100
daily_aggregates['idle_time_percentage'] = (daily_aggregates['idle_minutes'] / daily_aggregates['used_minutes']) * 100
daily_aggregates['peak_hour_utilization'] = daily_aggregates['peak_hour_sessions']

# Schritt 2: DataFrame für die Zeitreihen-Analyse vorbereiten
kpi_df = daily_aggregates.reset_index()[['hour', 'utilization_rate', 'idle_time_percentage', 'peak_hour_utilization']]
kpi_df.head()

# Plot 1: Utilization Rate
plt.figure(figsize=(12, 6))
sns.lineplot(data=kpi_df, x='hour', y='utilization_rate', marker='o', color='tab:blue')
plt.title('Utilization Rate Over Time')
plt.xlabel('Date')
plt.ylabel('Utilization Rate (%)')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Plot 2: Idle Time Percentage
plt.figure(figsize=(12, 6))
sns.lineplot(data=kpi_df, x='hour', y='idle_time_percentage', marker='o', color='tab:orange')
plt.title('Idle Time Percentage Over Time')
plt.xlabel('Date')
plt.ylabel('Idle Time Percentage (%)')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()

# Plot 3: Peak Hour Utilization
plt.figure(figsize=(12, 6))
sns.lineplot(data=kpi_df, x='hour', y='peak_hour_utilization', marker='o', color='green')
plt.title('Peak Hour Utilization Over Time')
plt.xlabel('Date')
plt.ylabel('Peak Hour Sessions')
plt.grid(visible=True, linestyle='--', alpha=0.6)
plt.tight_layout()
plt.show()


## ToDo hourly
## Varify KPIs
## show values